## **Trabajo Práctico Individual n° 2: Aprendizaje Supervisado**
____
__Universidad Tecnológica Nacional, Buenos Aires__\
__Ingeniería Industrial__\
__Ciencia de Datos - Curso I5521__
____

- Nombre: Candela
- Apellido: Abatedaga
- Legajo: 1673117

En este trabajo práctico aplicaremos algunas técnicas aprendidas para el aprendizaje automático supervisado.

Utilizaremos un dataset de airbnb donde buscaremos predecir el precio de las noches de cada hogar en funcion de algunas variables.

#### Variables:
- Unnamed: 0: Variable de contador de registros (no será utilizada en nuetro análisis)
- neighbourhood: Barrio donde se encuentra ubicado la casa.
- room_type: Tipo de hogar en alquiler.
- price: Precio por noche.
- minimum_nights: Cantidad minimas de noches que pueden alquilar el hogar.
- number_of_reviews: Cantidad de reviews
- reviews_per_month: Tasa de reviews que tienen por mes
- calculated_host_listings_count: Cantidad de hogares ofrecidos por el host.
- availability_365: Cantidad de dias que se encuentra en alquiler el hogar

## **Objetivos del trabajo practico**
- resolver los requisitos que se detallan en las proximas celdas
- dejar comentarios # en cada linea de codigo que se agregue
- fecha limite de entrega: 20 de Octubre 2022 a las 23h59

In [1]:
## Importo librerías de maniupulación de datos
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
## Importo librerías de Aprendizaje automático
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV

In [3]:
## Monto el drive, ya que el notebook será corrido en colaboratory.
from google.colab import drive
drive.mount('/content/gdrive') 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


##### **Objetivo 1**
Lo primero que deberan hacer será importar
- el dataset bajo en nombre "london"
- corroborar que lo hayamos cargado correctamente.

In [4]:
## Importo el dataset bajo el nombre "london".
london = pd.read_csv('/content/gdrive/MyDrive/Nivel V/Ciencia de Datos/TP/2/TP2_airbnb_london.csv')
## Corroboro haberlo cargado correctamente.
london.head()

,Unnamed: 0,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,0,Haringey,Private room,35,1,131,1.71,4,262
1,1,Ealing,Private room,70,2,2,0.07,6,62
2,2,Islington,Private room,45,1,14,0.14,2,364
3,5,Tower Hamlets,Entire home/apt,102,5,7,0.07,56,295
4,6,Barnet,Private room,29,5,114,1.13,3,285


##### **Objetivo 2**
Ahora imprimir también las dimensiones del dataset y eliminar la variable "Unnamed:0" del dataset.
Volver a guardar el dataset sin esta variable bajo el nombre "london" 

In [5]:
## Imprimo las dimensiones del dataset.
print(f'Dimensiones del dataset: {london.shape}')

Dimensiones del dataset: (51827, 9)


In [6]:
## Elimino muestras para poder correr en un tiempo lógico este notebook.
london = london.drop(london.index[5000:])
london.shape

(5000, 9)

In [7]:
## Elimino la variable "Unnamed: 0" del dataset y vuelvo a guardar el dataset sin dicha variable bajo el nombre "london"
london = london.drop(columns=['Unnamed: 0'])

In [8]:
## Imprimo las nuevas dimensiones del dataset y el head del dataset para verificar la eliminición.
print(f'Dimensiones del dataset: {london.shape}')

london.head()

Dimensiones del dataset: (5000, 8)


,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Haringey,Private room,35,1,131,1.71,4,262
1,Ealing,Private room,70,2,2,0.07,6,62
2,Islington,Private room,45,1,14,0.14,2,364
3,Tower Hamlets,Entire home/apt,102,5,7,0.07,56,295
4,Barnet,Private room,29,5,114,1.13,3,285


##### **Objetivo 3**
Corroborar que el dataset no cuenta con valores nulos, si los hubiera llenarlos todos con 0.

In [9]:
## Evalúo si hay valores nulos presentes en el dataset.
has_nans = london.isnull().values.any()

In [10]:
## Si hubiera valores nulos presentes en el dataset los lleno con 0.
if has_nans:
  london.fillna(0)

##### **Objetivo 4**
Mostrar el tipo de dato para las variables "neighbourhood" y "room_type".

In [11]:
## Muestro el tipo de dato para la variable "room_type".
london['room_type'].dtypes

dtype('O')

In [12]:
## Muestro el tipo de dato para la variable "neighbourhood".
london['neighbourhood'].dtypes

dtype('O')

##### Dividiremos nuestro dataset en la variable dependiente "y" (label o etiqueta) y las variables independientes "x"

In [13]:
## Divido el dataset en la variable dependiente "y", en este caso "price", y las independientes "x".
y = np.array(london[["price"]])
x = london.drop(['price'], axis=1)

##### **Objetivo 7**

Dividir el dataset entre Train y Test, tomar un test_size del 30%.

In [14]:
## Separo el dataset en train y test. Corresponderán a test el 30% de las muestras.
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3, random_state=4)

##### **Objetivo 5**
Generar las variables dummies para estas dos variables categóricas y unirlas al dataset.


In [15]:
## Genero las dummies para la variable neighbourhood tanto para el set de train como para el de test.
neigh_dummies_train = pd.get_dummies(xtrain['neighbourhood'])
neigh_dummies_test = pd.get_dummies(xtest['neighbourhood'])

In [16]:
## Genero las dummies para la variable room_type tanto para el set de train como para el de test.
room_dummies_train = pd.get_dummies(xtrain['room_type'])
room_dummies_test = pd.get_dummies(xtest['room_type'])

Imprimir las dimensiones y primeras lineas de estas dummies

In [17]:
## Imprimo las dimensiones y las primeras 5 líneas de las dummies generadas para la variable neighbourhood del set de train.
print(f'Dimensiones de neigh_dummies_train: {neigh_dummies_train.shape}')
neigh_dummies_train.head()

Dimensiones de neigh_dummies_train: (3500, 33)


,Barking and Dagenham,Barnet,Bexley,Brent,Bromley,Camden,City of London,Croydon,Ealing,Enfield,...,Merton,Newham,Redbridge,Richmond upon Thames,Southwark,Sutton,Tower Hamlets,Waltham Forest,Wandsworth,Westminster
2599,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3847,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1012,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3714,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3080,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [18]:
## Imprimo las dimensiones y las primeras 5 líneas de las dummies generadas para la variable neighbourhood del set de test.
print(f'Dimensiones de neigh_dummies_test: {neigh_dummies_test.shape}')
neigh_dummies_test.head()

Dimensiones de neigh_dummies_test: (1500, 33)


,Barking and Dagenham,Barnet,Bexley,Brent,Bromley,Camden,City of London,Croydon,Ealing,Enfield,...,Merton,Newham,Redbridge,Richmond upon Thames,Southwark,Sutton,Tower Hamlets,Waltham Forest,Wandsworth,Westminster
2175,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3156,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
337,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
444,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2334,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [19]:
## Imprimo las dimensiones y las primeras 5 líneas de las dummies generadas para la variable room_type del set de train.
print(f'Dimensiones de room_dummies_train: {room_dummies_train.shape}')
room_dummies_train.head(5)

Dimensiones de room_dummies_train: (3500, 3)


,Entire home/apt,Private room,Shared room
2599,0,1,0
3847,0,1,0
1012,0,1,0
3714,1,0,0
3080,0,1,0


In [20]:
## Imprimo las dimensiones y las primeras 5 líneas de las dummies generadas para la variable room_type del set de test.
print(f'Dimensiones de room_dummies_train: {room_dummies_test.shape}')
room_dummies_test.head(5)

Dimensiones de room_dummies_train: (1500, 3)


,Entire home/apt,Private room,Shared room
2175,0,1,0
3156,0,1,0
337,1,0,0
444,0,1,0
2334,1,0,0


In [21]:
## Elimino las variables neighbourhood y room_type del train y test set.
xtrain = xtrain.drop(columns=['neighbourhood','room_type'])
xtest = xtest.drop(columns=['neighbourhood','room_type'])

##### **Objetivo 8**
Con el StandardScaler generado en la linea siguiente, realizar el transform para el xtrain y xtest guardandolos en nuevas variables llamadas "xtrain_scal" y "xtest_scal".


In [22]:
scaler = preprocessing.StandardScaler().fit(xtrain)

In [23]:
## Transformo los datos de "xtrain" y los guardo en una nueva variabel llamada "xtrain_scal".
xtrain_scal = scaler.transform(xtrain)

In [24]:
## Transformo los datos de "xtest" y los guardo en una nueva variabel llamada "xtest_scal".
xtest_scal = scaler.transform(xtest)

In [25]:
type(xtest_scal)

numpy.ndarray

In [26]:
## Uno las dummies al los sets de train y test ya escaldos.
xtrain_scal = np.concatenate((xtrain_scal, neigh_dummies_train, room_dummies_train), axis=1)
xtest_scal = np.concatenate((xtest_scal, neigh_dummies_test, room_dummies_test), axis=1)

In [27]:
## Imprimo las dimensiones los sets de datos luego del procesamiento realizado.
print(f'Dimensiones del set de train: {xtrain_scal.shape}')
print(f'Dimensiones del set de test: {xtest_scal.shape}')

Dimensiones del set de train: (3500, 41)
Dimensiones del set de test: (1500, 41)


##### **Objetivo 9**
Crear un modelo lineal y entrenarlo para nuestro dataset.

Obtener las predicciones y el "mean_squared_error" para el test set.

In [28]:
## Creo un modelo lineal.
lin_reg = LinearRegression()

## Ajusto el modelo a las muestras de training.
lin_reg.fit(xtrain_scal, ytrain)

## Obtengo las predicciones a partir de mi modelo lineal entrenado.
ypred_lr = lin_reg.predict(xtest_scal)

## Imprimo dichas predicciones.
print(ypred_lr)

[[ 47.16015625]
 [ 55.18164062]
 [137.79882812]
 ...
 [ 70.4296875 ]
 [119.49414062]
 [148.66210938]]


In [29]:
## Calculo e imprimo el MSE.
mse_lr = mean_squared_error(ytest,ypred_lr)
mse_lr

1409.2307271016439

In [30]:
## Calculo e imprimo R^2.
r2_lr = r2_score(ytest,ypred_lr)
r2_lr

0.5160198066193075

##### **Objetivo 10**
Crear un SVM Regressor y un GridSearch para realizar la búsqueda de hiperparámetros.

Asignar dos posibles valores para el parámetro "kernel", y tres posibles valores para "C" y "gamma".

Setear al parámetro "refit" en True y hacer 5 CrossValidation.

In [31]:
## Creo el modelo.
mod_svmreg = SVR()

## Defino los hiperparámetros.
hyperparams = {'kernel':('sigmoid', 'rbf'),
              'C':[10, 100, 500],
              'gamma':[0.001, 0.1, 1]}

## Grid Search y Cross Validation para encontrar los mejores parámetros.
svmreg = GridSearchCV(mod_svmreg,
                   param_grid = hyperparams,
                   refit = True, 
                   cv = 5)

## Ajusto el modelo a las muestras de training.
svmreg.fit(xtrain_scal, ytrain.ravel())

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [10, 100, 500], 'gamma': [0.001, 0.1, 1],
                         'kernel': ('sigmoid', 'rbf')})

In [32]:
## Obtengo las predicciones a partir de mi modelo entrenado.
ypred_svmreg = svmreg.predict(xtest_scal)
## Imprimo dichas predicciones.
print(ypred_svmreg)

[ 35.77834446  42.64468972 103.25102963 ...  72.69980446  88.5070945
 132.7191187 ]


##### **Objetivo 11**

Mostrar cuales fueron los mejores hiperparámetros y para estos obtener el mean_squared_error en el test set.

In [33]:
## Muestro cuales fueron los mejores hiperparámetros.
print("Los mejores hiperparámetros fueron %s con un score del %0.4f" % (svmreg.best_params_, svmreg.best_score_))

Los mejores hiperparámetros fueron {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'} con un score del 0.4987


In [34]:
## Calculo e imprimo el MSE.
mse_svmreg = mean_squared_error(ytest,ypred_svmreg)
mse_svmreg

1446.3059032288056

##### **Objetivo 12**
Repetir el objetivo 10 pero ahora seteando nuevos valores para los hiperparámetros mencionados con el objetivo de mejorar las predicciones.

Cuales son los mejores hiperparámetros ahora?
Obtenemos una mejora en las predicciones?
Mostrar los resultados.

In [35]:
## Creo el nuevo modelo.
modelo_svmreg_1 = SVR()

## Defino los nuevos hiperparámetros.
hyperparams = {'kernel':('sigmoid', 'rbf'),
              'C':[10, 100, 200],
              'gamma':[0.05, 0.1, 0.5]}

## Grid Search y Cross Validation para encontrar los mejores parámetros.
reg_1 = GridSearchCV(modelo_svmreg_1,
                   param_grid = hyperparams,
                   refit = True, 
                   cv = 5)

reg_1.fit(xtrain_scal, ytrain.ravel())

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [10, 100, 200], 'gamma': [0.05, 0.1, 0.5],
                         'kernel': ('sigmoid', 'rbf')})

In [36]:
## Obtengo las predicciones a partir de mi modelo entrenado.
ypred_svmreg_1 = reg_1.predict(xtest_scal)
## Imprimo dichas predicciones.
print(ypred_svmreg_1)

[ 37.0465891   46.22133961 111.36041911 ...  77.5851018   96.98516286
 127.450243  ]


In [37]:
## Muestro cuales fueron los mejores hiperparámetros en esta ocasión.
print("Los mejores hiperparámetros ahora fueron %s con un score del %0.4f" % (reg_1.best_params_, reg_1.best_score_))

Los mejores hiperparámetros ahora fueron {'C': 100, 'gamma': 0.05, 'kernel': 'rbf'} con un score del 0.5039


In [38]:
## Calculo e imprimo el MSE.
mse_svmreg_1 = mean_squared_error(ytest,ypred_svmreg_1)
mse_svmreg_1

1433.023821733112

In [39]:
## Comparo los MSE obtenidos.
if mse_svmreg_1 < mse_svmreg:
  print('El MSE se redujo con respecto al obtenido con el modelo anterior.') 
elif mse_svmreg_1 == mse_svmreg:
  print('El MSE es igual al obtenido con el modelo anterior.') 
else:
  print('El MSE se increnmentó con respecto al obtenido con el modelo anterior.')

El MSE se redujo con respecto al obtenido con el modelo anterior.


In [40]:
## Calculo el R^2 para ambos modelos. El R^2 explica la proporción de la varianza de “y” que explica el modelo de regresión.
r2_svmreg = r2_score(ytest,ypred_svmreg)
r2_svmreg_1 = r2_score(ytest, ypred_svmreg_1)
print(f'El modelo del Objetivo 10 explica {round(r2_svmreg * 100,3)}% de la varianza de "y". R2 = {round(r2_svmreg,3)}')
print(f'El modelo del Objetivo 12 explica {round(r2_svmreg_1 * 100,3)}% de la varianza de "y". R2 = {round(r2_svmreg_1,3)}')
print(f'El modelo del Objetivo 8 explica {round(r2_lr * 100,3)}% de la varianza de "y". R2 = {round(r2_lr,3)}')

El modelo del Objetivo 10 explica 50.329% de la varianza de "y". R2 = 0.503
El modelo del Objetivo 12 explica 50.785% de la varianza de "y". R2 = 0.508
El modelo del Objetivo 8 explica 51.602% de la varianza de "y". R2 = 0.516


En comparación con el modelo creado en el Objetivo 10, las predicciones mejoraron levemente con este último modelo.

El modelo lineal creado en el Objetvo 8 fue el que generó las mejores predicciones.